In [ ]:
import metmhn.Utilityfunctions as utils
import metmhn.regularized_optimization as reg_opt

import pandas as pd
import numpy as np
import jax as jax
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
import matplotlib.pyplot as plt
from sklearn.metrics import (roc_curve, roc_auc_score, 
                             precision_recall_curve)
from sklearn.linear_model import LogisticRegression
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

Read the data and split it in  training and test sets:

In [ ]:
mut_handle = "../data/paad/G12_PAADPANET_PM_z10_Events_30and5_Full.csv"
annot_handle = "../data/paad/G12_PAADPANET_PM_z10_sampleSelection.csv"

annot_data = pd.read_csv(annot_handle)
mut_data = pd.read_csv(mut_handle)
mut_data.rename(columns={"Unnamed: 0":"patientID"}, inplace = True)
dat = pd.merge(mut_data, annot_data.loc[:, ['patientID', 'metaStatus']], on=["patientID", "patientID"])

# Remove datapoints, that consist solely of NaNs
dat = dat.loc[dat.iloc[:,1:-3].isna().all(axis=1) == False, :]
dat = dat.loc[(dat.iloc[:,1:-3].sum(axis=1) > 0), :]

In [ ]:
# Split the dataset into a training and a test set
n_dat = dat.shape[0]
start = 'P.Mut.KRAS'
stop = 'M.Mut.PTPRT'
shuffled = dat.sample(frac=1)
train = shuffled.iloc[:int(0.8*n_dat), :]
test = shuffled.iloc[int(0.8*n_dat):, :]

train = train.set_index(["paired", "metaStatus"])
train = train.loc[zip(*[[0,0,0,1],["present", "absent", "isMetastasis", "isPaired"]]), start:stop]
train.loc[(0, ["present", "isMetastasis"]), "Seeding"] = 1
train.loc[(0, "absent"), "Seeding"] = 0
train.loc[(1, "isPaired"), "Seeding"] = 1
train = train.astype('int8')
train_po, train_mo, train_pm, train_c = utils.split_data(train)

test = test.set_index(["paired", "metaStatus"])
test = test.loc[zip(*[[0,0,0,1],["present", "absent", "isMetastasis", "isPaired"]]), start:stop]
test.loc[(0, ["present", "isMetastasis"]), "Seeding"] = 1
test.loc[(0, "absent"), "Seeding"] = 0
test.loc[(1, "isPaired"), "Seeding"] = 1
test = test.astype('int8')
test_po, test_mo, test_pm, test_c = utils.split_data(train)
test = np.row_stack((test_po, test_pm, test_c))

Learn the metMHN:

In [ ]:
penal1 = 1/train.shape[0] #L1 penalty on off-diagonals
m_p_corr = 0.83
th_init, fd_init, sd_init = utils.indep(jnp.array(train.to_numpy()), train_c.shape[0])
theta, fd_effects, sd_effects = reg_opt.learn_mhn(th_init, fd_init, sd_init, train_po, train_pm, 
                                                  train_mo, train_c, m_p_corr, penal1)

Calculate the probability that a MT is present given the PT-genotype:

In [ ]:
mhn_probs = reg_opt.p_unobs_seeding(theta, fd_effects, jnp.array(test))

Learn a standard logistic regression model as baseline :

In [ ]:
train_labels = train.loc[:, 'Seeding']
logReg = LogisticRegression(C=1, max_iter=1000)
logReg.fit(train.iloc[:,:-1].to_numpy(), train_labels)
reg_probs = logReg.predict_proba(test[:, :-1])

Compare both models:

In [ ]:
test_labels = test[:, -1]
mhn_auc = roc_auc_score(test_labels, mhn_probs)
mhn_fpr, mhn_tpr, _ = roc_curve(test_labels, mhn_probs)
mhn_precision, mhn_recall, _ = precision_recall_curve(test_labels, mhn_probs)

reg_auc = roc_auc_score(test_labels, reg_probs[:,1])
reg_fpr, reg_tpr, _ = roc_curve(test_labels, reg_probs[:, 1])
reg_precision, reg_recall, _ = precision_recall_curve(test_labels, reg_probs[:,1])

plt.plot(reg_recall, reg_precision, linestyle="--", label="LR")
plt.plot(mhn_recall, mhn_precision, linestyle="-", label="metMHN")
plt.legend()

In [ ]:
print(reg_auc, mhn_auc)